[[Keras] 利用Keras建構LSTM模型，以Stock Prediction 為例 1](https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b)

In [1]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # K折交叉验证模块
from sklearn.preprocessing import MinMaxScaler

import QUANTAXIS as QA
import pandas as pd
import numpy as np
import pyecharts
import talib

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,5]

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Noto Sans CJK SC','SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set(font=['Noto Sans CJK SC','SimHei'])

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import ExtraTreeRegressor

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [57]:
stock_code='601398'
benchmark_code='399300'
start_time='2005-01-01'
end_time='2018-12-31'

data_raw=QA.QA_fetch_stock_day_adv(stock_code, start_time, end_time).to_qfq().data.reset_index().set_index('date')
data_raw.head(1)

,code,open,high,low,close,volume,amount,preclose,adj
date,,,,,,,,,
2006-10-27,601398,1.992133,2.01557,1.910104,1.921823,4.407654e+07,8.725310e+09,NaN,0.585922


In [58]:
data=data_raw.drop(columns=['code','amount','preclose','adj'])
data.head(1)

,open,high,low,close,volume
date,,,,,
2006-10-27,1.992133,2.01557,1.910104,1.921823,4.407654e+07


### Augment Features
除了基本資料提供的Features(Open, High, Low, Close, Adj Close, Volume)以外，還可自己增加Features，例如星期幾、幾月、幾號等等。

In [59]:
def augFeatures(train):
    df=train.copy()
    df["year"] = df.index.year
    df["month"] = df.index.month
    df["date"] = df.index.day
    df["day"] = df.index.dayofweek
    return df

In [60]:
data=augFeatures(data)
data.head(1)

,open,high,low,close,volume,year,month,date,day
date,,,,,,,,,
2006-10-27,1.992133,2.01557,1.910104,1.921823,4.407654e+07,2006,10,27,4


### Normalization
將所有資料做正規化

In [62]:
def normalize(train):
    return train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))

In [63]:
data=normalize(data)
data.head(1)

,open,high,low,close,volume,year,month,date,day
date,,,,,,,,,
2006-10-27,-0.29094,-0.286786,-0.305092,-0.297378,0.947025,-0.53514,0.303308,0.362023,0.497282


### Build Training Data
輸入X_train: 利用前30天的Open, High, Low, Close, Volume, month, year, date, day作為Features，shape為(30, 9)

輸出Y_train: 利用未來5天的Close作為Features，shape為(5,1)

我們須將資料做位移的展開作為Training Data，如圖(1)所示。

![](https://cdn-images-1.medium.com/max/800/1*QUwzqa1Gi0bGbsIDQqpOhw.png)

In [64]:
def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["close"]))
    return np.array(X_train), np.array(Y_train)

### 資料亂序
將資料打散，而非照日期排序

In [65]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

### Training data & Validation data
將Training Data取一部份當作Validation Data

In [66]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [67]:
train = data_raw.drop(columns=['code','amount','preclose','adj'])

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
X_train.shape,Y_train.shape,X_val.shape,Y_val.shape

((2618, 30, 9), (2618, 5), (290, 30, 9), (290, 5))

### 模型建置

![](https://cdn-images-1.medium.com/max/800/1*bOOrk_un_DQ7k3THPeZpkA.jpeg)

![](https://cdn-images-1.medium.com/max/800/1*lCOYLu3M8tIUkNF-4ngXoA.png)

### 一對一模型
由於是一對一模型，因此return_sequences 也可設為False ，但Y_train 以及Y_val的維度需改為二維(2618,1)以及(290,1) 。

In [69]:
def buildOneToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

將過去的天數pastDay設為1，預測的天數futureDay也設為1

In [71]:
train = data_raw.drop(columns=['code','amount','preclose','adj'])
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 9))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 1, 10)             800       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________
Train on 2647 samples, validate on 294 samples
Epoch 1/1000
2647/2647 [==============================] - ETA: 18s - loss: 0.04 - 1s 401us/step - loss: 0.0414 - val_loss: 0.0393
Epoch 2/1000
2647/2647 [==============================] - ETA: 0s - loss: 0.034 - 0s 17us/step - loss: 0.0339 - val_loss: 0.0321
Epoch 3/1000
2647/2647 [==============================] - ETA: 0s - loss: 0.029 - 0s 15us/step - loss: 0.0275 - val_loss: 0.0257
Epoch 4/1000
2647/2647 [==============================] - ETA: 0s - loss: 0.023 - 0s 15us/step - loss: 0.

2647/2647 [==============================] - ETA: 0s - loss: 1.9849e-0 - 0s 14us/step - loss: 1.6961e-04 - val_loss: 1.5457e-04
Epoch 56/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.9825e-0 - 0s 15us/step - loss: 1.6977e-04 - val_loss: 1.5605e-04
Epoch 57/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.9595e-0 - 0s 14us/step - loss: 1.6902e-04 - val_loss: 1.5382e-04
Epoch 58/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.2437e-0 - 0s 15us/step - loss: 1.6871e-04 - val_loss: 1.5493e-04
Epoch 59/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.6481e-0 - 0s 14us/step - loss: 1.6870e-04 - val_loss: 1.5467e-04
Epoch 60/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.4874e-0 - 0s 15us/step - loss: 1.6940e-04 - val_loss: 1.5680e-04
Epoch 61/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.6085e-0 - 0s 14us/step - loss: 1.6879e-04 - val_loss: 1.5402e-04
Epoch 62/1000
2647/2

2647/2647 [==============================] - ETA: 0s - loss: 1.0888e-0 - ETA: 0s - loss: 1.5669e-0 - 0s 33us/step - loss: 1.6433e-04 - val_loss: 1.5061e-04
Epoch 113/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.4083e-0 - ETA: 0s - loss: 1.7548e-0 - 0s 34us/step - loss: 1.6302e-04 - val_loss: 1.4912e-04
Epoch 114/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.6647e-0 - 0s 15us/step - loss: 1.6365e-04 - val_loss: 1.4898e-04
Epoch 115/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.3769e-0 - 0s 15us/step - loss: 1.6375e-04 - val_loss: 1.5475e-04
Epoch 116/1000
2647/2647 [==============================] - ETA: 0s - loss: 2.4738e-0 - 0s 15us/step - loss: 1.6424e-04 - val_loss: 1.4895e-04
Epoch 117/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.5787e-0 - 0s 14us/step - loss: 1.6310e-04 - val_loss: 1.4841e-04
Epoch 118/1000
2647/2647 [==============================] - ETA: 0s - loss: 1.6417e-0 - 0s 16us/step 

### 多對一模型
LSTM參數return_sequences=False ，未設定時default也為False，而且不可使用TimeDistribution

In [72]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

需要設定的有pastDay=30、future=1 ，且注意Y_train 的維度需為二維

In [73]:
train = data_raw.drop(columns=['code','amount','preclose','adj'])
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 9))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10)                800       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________
Train on 2621 samples, validate on 291 samples
Epoch 1/1000
2621/2621 [==============================] - ETA: 14s - loss: 0.09 - ETA: 1s - loss: 0.0758 - ETA: 0s - loss: 0.069 - ETA: 0s - loss: 0.061 - 1s 387us/step - loss: 0.0562 - val_loss: 0.0323
Epoch 2/1000
2621/2621 [==============================] - ETA: 0s - loss: 0.029 - ETA: 0s - loss: 0.028 - ETA: 0s - loss: 0.026 - ETA: 0s - loss: 0.022 - 0s 82us/step - loss: 0.0217 - val_loss: 0.0143
Epoch 3/1000
2621/2621 [==============================] - ETA: 0s - loss: 0.011 - ETA: 0s

2621/2621 [==============================] - ETA: 0s - loss: 4.2651e-0 - ETA: 0s - loss: 4.1415e-0 - ETA: 0s - loss: 4.4560e-0 - 0s 68us/step - loss: 4.3267e-04 - val_loss: 4.9267e-04
Epoch 37/1000
2621/2621 [==============================] - ETA: 0s - loss: 3.0916e-0 - ETA: 0s - loss: 4.0657e-0 - ETA: 0s - loss: 4.2453e-0 - 0s 67us/step - loss: 4.2787e-04 - val_loss: 4.7106e-04
Epoch 38/1000
2621/2621 [==============================] - ETA: 0s - loss: 4.8645e-0 - ETA: 0s - loss: 3.9806e-0 - ETA: 0s - loss: 4.1627e-0 - ETA: 0s - loss: 4.1686e-0 - 0s 74us/step - loss: 4.2018e-04 - val_loss: 4.7445e-04
Epoch 39/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.8200e-0 - ETA: 0s - loss: 4.3501e-0 - ETA: 0s - loss: 4.2370e-0 - ETA: 0s - loss: 4.1591e-0 - 0s 76us/step - loss: 4.1369e-04 - val_loss: 4.5887e-04
Epoch 40/1000
2621/2621 [==============================] - ETA: 0s - loss: 6.1214e-0 - ETA: 0s - loss: 4.1064e-0 - ETA: 0s - loss: 4.1778e-0 - ETA: 0s - loss: 4.0148e

Epoch 74/1000
2621/2621 [==============================] - ETA: 0s - loss: 4.9478e-0 - ETA: 0s - loss: 3.5605e-0 - ETA: 0s - loss: 3.2923e-0 - ETA: 0s - loss: 3.1497e-0 - 0s 77us/step - loss: 3.0493e-04 - val_loss: 3.6500e-04
Epoch 75/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.6201e-0 - ETA: 0s - loss: 2.9022e-0 - ETA: 0s - loss: 2.9765e-0 - ETA: 0s - loss: 3.0543e-0 - 0s 72us/step - loss: 3.0464e-04 - val_loss: 3.6087e-04
Epoch 76/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.5646e-0 - ETA: 0s - loss: 2.7243e-0 - ETA: 0s - loss: 3.1167e-0 - ETA: 0s - loss: 3.0190e-0 - 0s 75us/step - loss: 3.0100e-04 - val_loss: 3.6091e-04
Epoch 77/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.5351e-0 - ETA: 0s - loss: 2.9229e-0 - ETA: 0s - loss: 2.8335e-0 - 0s 68us/step - loss: 3.0017e-04 - val_loss: 3.5817e-04
Epoch 78/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.3347e-0 - ETA: 0s - loss: 2.7615e-0 - ETA: 0s -

2621/2621 [==============================] - ETA: 0s - loss: 2.9859e-0 - ETA: 0s - loss: 2.8876e-0 - ETA: 0s - loss: 2.5300e-0 - ETA: 0s - loss: 2.5396e-0 - 0s 85us/step - loss: 2.5875e-04 - val_loss: 3.0810e-04
Epoch 111/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.6124e-0 - ETA: 0s - loss: 2.4113e-0 - ETA: 0s - loss: 2.5886e-0 - ETA: 0s - loss: 2.5780e-0 - 0s 79us/step - loss: 2.5930e-04 - val_loss: 3.1181e-04
Epoch 112/1000
2621/2621 [==============================] - ETA: 0s - loss: 3.6501e-0 - ETA: 0s - loss: 2.6554e-0 - ETA: 0s - loss: 2.7077e-0 - ETA: 0s - loss: 2.6488e-0 - 0s 80us/step - loss: 2.6058e-04 - val_loss: 3.0823e-04
Epoch 113/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.7529e-0 - ETA: 0s - loss: 2.5928e-0 - ETA: 0s - loss: 2.6961e-0 - ETA: 0s - loss: 2.6636e-0 - 0s 80us/step - loss: 2.5916e-04 - val_loss: 3.1206e-04
Epoch 114/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.9286e-0 - ETA: 0s - loss: 2.7

...

2621/2621 [==============================] - ETA: 0s - loss: 1.8767e-0 - ETA: 0s - loss: 1.8662e-0 - ETA: 0s - loss: 2.1243e-0 - ETA: 0s - loss: 2.1472e-0 - ETA: 0s - loss: 2.0733e-0 - 0s 95us/step - loss: 2.0670e-04 - val_loss: 2.5111e-04
Epoch 183/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.9855e-0 - ETA: 0s - loss: 2.0673e-0 - ETA: 0s - loss: 1.9982e-0 - ETA: 0s - loss: 2.0585e-0 - 0s 86us/step - loss: 2.0668e-04 - val_loss: 2.5197e-04
Epoch 184/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.3314e-0 - ETA: 0s - loss: 2.2410e-0 - ETA: 0s - loss: 2.1339e-0 - ETA: 0s - loss: 2.0379e-0 - 0s 79us/step - loss: 2.0523e-04 - val_loss: 2.5462e-04
Epoch 185/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.4373e-0 - ETA: 0s - loss: 1.8115e-0 - ETA: 0s - loss: 1.7257e-0 - ETA: 0s - loss: 1.9647e-0 - ETA: 0s - loss: 2.0342e-0 - 0s 95us/step - loss: 2.0313e-04 - val_loss: 2.4921e-04
Epoch 186/1000
2621/2621 [=========================

Epoch 218/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.4020e-0 - ETA: 0s - loss: 1.9408e-0 - ETA: 0s - loss: 1.8929e-0 - ETA: 0s - loss: 1.8954e-0 - 0s 79us/step - loss: 1.8691e-04 - val_loss: 2.2768e-04
Epoch 219/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.9352e-0 - ETA: 0s - loss: 1.8817e-0 - ETA: 0s - loss: 1.8949e-0 - ETA: 0s - loss: 1.9194e-0 - 0s 88us/step - loss: 1.8475e-04 - val_loss: 2.3588e-04
Epoch 220/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.7863e-0 - ETA: 0s - loss: 1.5890e-0 - ETA: 0s - loss: 1.6986e-0 - ETA: 0s - loss: 1.8765e-0 - 0s 79us/step - loss: 1.8856e-04 - val_loss: 2.3098e-04
Epoch 221/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.0952e-0 - ETA: 0s - loss: 1.7646e-0 - ETA: 0s - loss: 1.7552e-0 - ETA: 0s - loss: 1.7564e-0 - 0s 91us/step - loss: 1.8550e-04 - val_loss: 2.3436e-04
Epoch 222/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.6018e-0 - ETA:

2621/2621 [==============================] - ETA: 0s - loss: 1.1600e-0 - ETA: 0s - loss: 1.5920e-0 - ETA: 0s - loss: 1.7401e-0 - ETA: 0s - loss: 1.6586e-0 - 0s 78us/step - loss: 1.6959e-04 - val_loss: 2.0736e-04
Epoch 255/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.8419e-0 - ETA: 0s - loss: 1.7766e-0 - ETA: 0s - loss: 1.7166e-0 - ETA: 0s - loss: 1.6383e-0 - 0s 90us/step - loss: 1.7289e-04 - val_loss: 2.0476e-04
Epoch 256/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.6285e-0 - ETA: 0s - loss: 1.8699e-0 - ETA: 0s - loss: 1.6699e-0 - ETA: 0s - loss: 1.7152e-0 - 0s 84us/step - loss: 1.6913e-04 - val_loss: 1.9981e-04
Epoch 257/1000
2621/2621 [==============================] - ETA: 0s - loss: 2.2594e-0 - ETA: 0s - loss: 1.7549e-0 - ETA: 0s - loss: 1.7727e-0 - ETA: 0s - loss: 1.7934e-0 - 0s 88us/step - loss: 1.7061e-04 - val_loss: 2.0211e-04
Epoch 258/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.1822e-0 - ETA: 0s - loss: 1.6

Epoch 290/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.3966e-0 - ETA: 0s - loss: 1.2824e-0 - ETA: 0s - loss: 1.4684e-0 - ETA: 0s - loss: 1.5947e-0 - 0s 81us/step - loss: 1.6064e-04 - val_loss: 2.0101e-04
Epoch 291/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.4399e-0 - ETA: 0s - loss: 1.6275e-0 - ETA: 0s - loss: 1.6479e-0 - ETA: 0s - loss: 1.6608e-0 - 0s 82us/step - loss: 1.6265e-04 - val_loss: 1.8337e-04
Epoch 292/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.7208e-0 - ETA: 0s - loss: 1.6653e-0 - ETA: 0s - loss: 1.6524e-0 - ETA: 0s - loss: 1.6146e-0 - 0s 83us/step - loss: 1.6300e-04 - val_loss: 1.8475e-04
Epoch 293/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.6599e-0 - ETA: 0s - loss: 1.6951e-0 - ETA: 0s - loss: 1.6665e-0 - ETA: 0s - loss: 1.5440e-0 - 0s 81us/step - loss: 1.5803e-04 - val_loss: 1.8557e-04
Epoch 294/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.0537e-0 - ETA:

Epoch 327/1000
2621/2621 [==============================] - ETA: 0s - loss: 1.0758e-0 - ETA: 0s - loss: 1.4554e-0 - ETA: 0s - loss: 1.5352e-0 - ETA: 0s - loss: 1.5375e-0 - 0s 74us/step - loss: 1.5342e-04 - val_loss: 1.8129e-04
Epoch 00327: early stopping


### 一對多模型
因為是一對多模型Timesteps只有1，因此return_sequences=False 才可執行

In [74]:
def buildOneToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
    # output shape: (5, 1)
    model.add(Dense(1))
    model.add(RepeatVector(5))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

將pastDay 設為1, futureDay 設為5

In [75]:
train = data_raw.drop(columns=['code','amount','preclose','adj'])
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 9))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10)                800       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________
Train on 2644 samples, validate on 293 samples
Epoch 1/1000
2644/2644 [==============================] - ETA: 17s - loss: 0.05 - 1s 377us/step - loss: 0.0421 - val_loss: 0.0330
Epoch 2/1000
2644/2644 [==============================] - ETA: 0s - loss: 0.040 - 0s 17us/step - loss: 0.0345 - val_loss: 0.0270
Epoch 3/1000
2644/2644 [==============================] - ETA: 0s - loss: 0.023 - 0s 18us/step - loss:

Epoch 55/1000
2644/2644 [==============================] - ETA: 0s - loss: 5.0226e-0 - 0s 15us/step - loss: 4.4949e-04 - val_loss: 3.1567e-04
Epoch 56/1000
2644/2644 [==============================] - ETA: 0s - loss: 3.8920e-0 - 0s 14us/step - loss: 4.5042e-04 - val_loss: 3.1992e-04
Epoch 57/1000
2644/2644 [==============================] - ETA: 0s - loss: 5.3401e-0 - 0s 15us/step - loss: 4.4955e-04 - val_loss: 3.1474e-04
Epoch 58/1000
2644/2644 [==============================] - ETA: 0s - loss: 6.0013e-0 - 0s 15us/step - loss: 4.5002e-04 - val_loss: 3.1478e-04
Epoch 59/1000
2644/2644 [==============================] - ETA: 0s - loss: 3.7781e-0 - 0s 14us/step - loss: 4.5007e-04 - val_loss: 3.1507e-04
Epoch 60/1000
2644/2644 [==============================] - ETA: 0s - loss: 3.3784e-0 - 0s 15us/step - loss: 4.4975e-04 - val_loss: 3.1625e-04
Epoch 61/1000
2644/2644 [==============================] - ETA: 0s - loss: 6.1759e-0 - 0s 15us/step - loss: 4.4834e-04 - val_loss: 3.1376e-04
Epoch 

Epoch 113/1000
2644/2644 [==============================] - ETA: 0s - loss: 3.0911e-0 - 0s 20us/step - loss: 4.4452e-04 - val_loss: 3.0774e-04
Epoch 114/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.7252e-0 - 0s 16us/step - loss: 4.4652e-04 - val_loss: 3.0839e-04
Epoch 115/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.2530e-0 - 0s 16us/step - loss: 4.4378e-04 - val_loss: 3.0790e-04
Epoch 116/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.4228e-0 - 0s 16us/step - loss: 4.4300e-04 - val_loss: 3.0722e-04
Epoch 117/1000
2644/2644 [==============================] - ETA: 0s - loss: 5.1890e-0 - 0s 14us/step - loss: 4.4282e-04 - val_loss: 3.0802e-04
Epoch 118/1000
2644/2644 [==============================] - ETA: 0s - loss: 5.4918e-0 - 0s 14us/step - loss: 4.4388e-04 - val_loss: 3.0742e-04
Epoch 119/1000
2644/2644 [==============================] - ETA: 0s - loss: 6.8873e-0 - 0s 14us/step - loss: 4.4298e-04 - val_loss: 3.0952e-04

2644/2644 [==============================] - ETA: 0s - loss: 5.7088e-0 - 0s 13us/step - loss: 4.4052e-04 - val_loss: 3.0523e-04
Epoch 171/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.9846e-0 - 0s 18us/step - loss: 4.3787e-04 - val_loss: 3.0453e-04
Epoch 172/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.1920e-0 - 0s 16us/step - loss: 4.3774e-04 - val_loss: 3.0531e-04
Epoch 173/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.8203e-0 - 0s 13us/step - loss: 4.3719e-04 - val_loss: 3.0643e-04
Epoch 174/1000
2644/2644 [==============================] - ETA: 0s - loss: 3.5730e-0 - 0s 11us/step - loss: 4.3928e-04 - val_loss: 3.0456e-04
Epoch 175/1000
2644/2644 [==============================] - ETA: 0s - loss: 4.3830e-0 - 0s 17us/step - loss: 4.3718e-04 - val_loss: 3.0455e-04
Epoch 176/1000
2644/2644 [==============================] - ETA: 0s - loss: 5.3980e-0 - 0s 16us/step - loss: 4.3749e-04 - val_loss: 3.0444e-04
Epoch 177/1000

### 多對多模型 (輸入與輸出相同長度)

將return_sequences 設為True ，再用TimeDistributed(Dense(1)) 將輸出調整為(5,1)

In [76]:
def buildManyToManyModel(shape):
    model = Sequential()
    model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
    # output shape: (5, 1)
    model.add(TimeDistributed(Dense(1)))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

將pastDay 以及futureDay 設為相同長度5

In [77]:
train = data_raw.drop(columns=['code','amount','preclose','adj'])
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 9))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 5, 10)             800       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1)              11        
Total params: 811
Trainable params: 811
Non-trainable params: 0
_________________________________________________________________
Train on 2640 samples, validate on 293 samples
Epoch 1/1000
2640/2640 [==============================] - ETA: 17s - loss: 0.06 - ETA: 0s - loss: 0.0498 - 1s 395us/step - loss: 0.0482 - val_loss: 0.0419
Epoch 2/1000
2640/2640 [==============================] - ETA: 0s - loss: 0.031 - ETA: 0s - loss: 0.032 - 0s 27us/step - loss: 0.0320 - val_loss: 0.0275
Epoch 3/1000
2640/2640 [==============================] - ETA: 0s - loss: 0.025 - ETA: 0s - loss: 0.021 - 0s 29us/step - loss: 0.0204 - val_loss: 0.0168
Epoch 4/1000
2640/2640 [========

2640/2640 [==============================] - ETA: 0s - loss: 6.9608e-0 - ETA: 0s - loss: 8.0318e-0 - 0s 29us/step - loss: 8.1100e-04 - val_loss: 8.2247e-04
Epoch 49/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.1296e-0 - ETA: 0s - loss: 8.0051e-0 - 0s 26us/step - loss: 8.0467e-04 - val_loss: 8.1558e-04
Epoch 50/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.8597e-0 - ETA: 0s - loss: 8.0393e-0 - 0s 27us/step - loss: 7.9973e-04 - val_loss: 8.1101e-04
Epoch 51/1000
2640/2640 [==============================] - ETA: 0s - loss: 8.2273e-0 - ETA: 0s - loss: 7.9029e-0 - 0s 27us/step - loss: 7.9261e-04 - val_loss: 8.0265e-04
Epoch 52/1000
2640/2640 [==============================] - ETA: 0s - loss: 9.5178e-0 - ETA: 0s - loss: 7.8065e-0 - 0s 27us/step - loss: 7.8728e-04 - val_loss: 7.9954e-04
Epoch 53/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.5801e-0 - ETA: 0s - loss: 7.6372e-0 - 0s 28us/step - loss: 7.8289e-04 - val_loss: 7.922

2640/2640 [==============================] - ETA: 0s - loss: 6.1441e-0 - ETA: 0s - loss: 6.7348e-0 - 0s 28us/step - loss: 6.8568e-04 - val_loss: 6.9519e-04
Epoch 97/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.6144e-0 - ETA: 0s - loss: 6.9574e-0 - 0s 27us/step - loss: 6.8614e-04 - val_loss: 6.9174e-04
Epoch 98/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.8923e-0 - ETA: 0s - loss: 6.7858e-0 - 0s 26us/step - loss: 6.8423e-04 - val_loss: 6.9230e-04
Epoch 99/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.6761e-0 - ETA: 0s - loss: 6.9774e-0 - 0s 26us/step - loss: 6.8610e-04 - val_loss: 6.9178e-04
Epoch 100/1000
2640/2640 [==============================] - ETA: 0s - loss: 9.3960e-0 - ETA: 0s - loss: 7.1287e-0 - 0s 29us/step - loss: 6.8701e-04 - val_loss: 6.9279e-04
Epoch 101/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.5278e-0 - ETA: 0s - loss: 6.6142e-0 - 0s 28us/step - loss: 6.8269e-04 - val_loss: 6.8

Epoch 144/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.4550e-0 - ETA: 0s - loss: 6.7259e-0 - 0s 29us/step - loss: 6.6659e-04 - val_loss: 6.7762e-04
Epoch 145/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.7537e-0 - ETA: 0s - loss: 6.7986e-0 - 0s 28us/step - loss: 6.6520e-04 - val_loss: 6.8166e-04
Epoch 146/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.1694e-0 - ETA: 0s - loss: 6.7533e-0 - 0s 29us/step - loss: 6.6528e-04 - val_loss: 6.7840e-04
Epoch 147/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.7428e-0 - ETA: 0s - loss: 6.7299e-0 - 0s 27us/step - loss: 6.6641e-04 - val_loss: 6.8948e-04
Epoch 148/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.3013e-0 - ETA: 0s - loss: 6.5591e-0 - 0s 29us/step - loss: 6.6466e-04 - val_loss: 6.7857e-04
Epoch 149/1000
2640/2640 [==============================] - ETA: 0s - loss: 8.0610e-0 - ETA: 0s - loss: 6.8733e-0 - 0s 30us/step - loss: 6.6633e-

...

Epoch 288/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.1198e-0 - ETA: 0s - loss: 6.2273e-0 - 0s 32us/step - loss: 6.2075e-04 - val_loss: 6.6364e-04
Epoch 289/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.2401e-0 - ETA: 0s - loss: 6.0977e-0 - 0s 32us/step - loss: 6.1484e-04 - val_loss: 6.6512e-04
Epoch 290/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.2434e-0 - ETA: 0s - loss: 6.1335e-0 - 0s 30us/step - loss: 6.1515e-04 - val_loss: 6.6568e-04
Epoch 291/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.7124e-0 - ETA: 0s - loss: 6.0240e-0 - 0s 30us/step - loss: 6.1783e-04 - val_loss: 6.6593e-04
Epoch 292/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.6566e-0 - ETA: 0s - loss: 6.4297e-0 - 0s 30us/step - loss: 6.2118e-04 - val_loss: 6.6899e-04
Epoch 293/1000
2640/2640 [==============================] - ETA: 0s - loss: 7.5650e-0 - ETA: 0s - loss: 6.1978e-0 - 0s 30us/step - loss: 6.1507e-

Epoch 336/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.5328e-0 - ETA: 0s - loss: 6.0778e-0 - 0s 29us/step - loss: 6.1154e-04 - val_loss: 6.6365e-04
Epoch 337/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.7536e-0 - ETA: 0s - loss: 6.2533e-0 - 0s 29us/step - loss: 6.0733e-04 - val_loss: 6.5727e-04
Epoch 338/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.7124e-0 - ETA: 0s - loss: 5.8352e-0 - 0s 30us/step - loss: 6.0499e-04 - val_loss: 6.5524e-04
Epoch 339/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.4019e-0 - ETA: 0s - loss: 6.0896e-0 - 0s 30us/step - loss: 6.0469e-04 - val_loss: 6.5757e-04
Epoch 340/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.8996e-0 - ETA: 0s - loss: 6.0582e-0 - 0s 31us/step - loss: 6.0716e-04 - val_loss: 6.5756e-04
Epoch 341/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.2398e-0 - ETA: 0s - loss: 6.1767e-0 - 0s 29us/step - loss: 6.1389e-

Epoch 384/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.3179e-0 - ETA: 0s - loss: 6.2135e-0 - 0s 31us/step - loss: 5.9675e-04 - val_loss: 6.4818e-04
Epoch 385/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.9366e-0 - ETA: 0s - loss: 5.7127e-0 - 0s 32us/step - loss: 5.9317e-04 - val_loss: 6.5131e-04
Epoch 386/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.0057e-0 - ETA: 0s - loss: 5.7648e-0 - 0s 30us/step - loss: 5.9353e-04 - val_loss: 6.4776e-04
Epoch 387/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.2237e-0 - ETA: 0s - loss: 5.9934e-0 - 0s 31us/step - loss: 5.9545e-04 - val_loss: 6.4882e-04
Epoch 388/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.6566e-0 - ETA: 0s - loss: 5.9908e-0 - 0s 31us/step - loss: 5.9611e-04 - val_loss: 6.4850e-04
Epoch 389/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.1443e-0 - ETA: 0s - loss: 5.9417e-0 - 0s 30us/step - loss: 5.9735e-

Epoch 432/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.2214e-0 - ETA: 0s - loss: 5.8427e-0 - 0s 31us/step - loss: 5.7951e-04 - val_loss: 6.3568e-04
Epoch 433/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.5160e-0 - ETA: 0s - loss: 5.7980e-0 - 0s 29us/step - loss: 5.8015e-04 - val_loss: 6.3950e-04
Epoch 434/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.0040e-0 - ETA: 0s - loss: 5.8329e-0 - 0s 26us/step - loss: 5.8023e-04 - val_loss: 6.3696e-04
Epoch 435/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.5161e-0 - ETA: 0s - loss: 5.7033e-0 - 0s 27us/step - loss: 5.8198e-04 - val_loss: 6.3588e-04
Epoch 436/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.2038e-0 - ETA: 0s - loss: 5.9035e-0 - 0s 27us/step - loss: 5.8190e-04 - val_loss: 6.3839e-04
Epoch 437/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.1581e-0 - ETA: 0s - loss: 5.8361e-0 - 0s 29us/step - loss: 5.8044e-

Epoch 480/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.7384e-0 - ETA: 0s - loss: 5.5209e-0 - 0s 29us/step - loss: 5.6604e-04 - val_loss: 6.1677e-04
Epoch 481/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.1916e-0 - ETA: 0s - loss: 5.7754e-0 - 0s 27us/step - loss: 5.6505e-04 - val_loss: 6.1529e-04
Epoch 482/1000
2640/2640 [==============================] - ETA: 0s - loss: 4.7553e-0 - ETA: 0s - loss: 5.6438e-0 - 0s 31us/step - loss: 5.6530e-04 - val_loss: 6.2088e-04
Epoch 483/1000
2640/2640 [==============================] - ETA: 0s - loss: 6.9782e-0 - ETA: 0s - loss: 5.9429e-0 - 0s 34us/step - loss: 5.7339e-04 - val_loss: 6.2078e-04
Epoch 484/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.0180e-0 - ETA: 0s - loss: 5.7838e-0 - 0s 31us/step - loss: 5.6662e-04 - val_loss: 6.1405e-04
Epoch 485/1000
2640/2640 [==============================] - ETA: 0s - loss: 5.7396e-0 - ETA: 0s - loss: 5.5173e-0 - 0s 31us/step - loss: 5.6430e-

### Reference
1. [李弘毅 — 機器學習 RNN](http://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2017/Lecture/RNN.pdf)

2. [Keras關於LSTM的units參數，還是不理解?](https://www.zhihu.com/question/64470274)

3. [Many to one and many to many LSTM examples in Keras](https://stackoverflow.com/questions/43034960/many-to-one-and-many-to-many-lstm-examples-in-keras)

4. [Wiki — 長短期記憶](https://zh.wikipedia.org/wiki/%E9%95%B7%E7%9F%AD%E6%9C%9F%E8%A8%98%E6%86%B6)